# ADM - Homework #5

### Dependencies

In [2]:
import numpy as np
import pandas as pd

## 1. Data

## 2. Implementation of the backend

### Functionality 1 - Get the overall features of the graph

###  Functionality 2 - Find the best users! 

### Functionality 3 - Shortest Ordered Route 

### Functionality 4 - Disconnecting graphs 

## 3. Implementation of the frontend

### Visualization 1 - Visualize the overall features of the graph 

### Visualization 2 - Visualize the best user! 

### Visualization 3 - Visualize the Shortest Ordered Route 

### Visualization 4 - Visualize disconnecting graphs 

## 4. Algorithmic question